In [64]:
import importlib

import numpy as np
import json
import os

from cocoView import *

In [219]:
image_dir = './testCOCO/filtered/test/'
annotation_path = './testCOCO/filtered/test/_annotations.coco.json'

coco_dataset = CocoDataset(annotation_path, image_dir)
# coco_dataset.display_info()
# coco_dataset.display_licenses()
coco_dataset.display_categories()

html = coco_dataset.display_image(21)
IPython.display.HTML(html)

Categories:
  super_category: none
    id 0: Plastic

  super_category: Plastic
    id 1: 1
    id 2: 2
    id 3: 3
    id 4: 3 0 0 0 0



In [218]:
image_dir = './testCOCO/filtered/cutous/'
# extractCOCOparticles(annotation_path, image_dir, save_data_path)
# TODO morphology, better contour -> generate



In [163]:

            
folder='train'
pth2json='./testCOCO/'+folder+'/_annotations.coco.json'
data,imgPaths = delNoSegJSONelem(pth2json)
print(len(imgPaths)) # test imgs = 30; train imgs = 2304
with open("./testCOCO/filtered/"+folder+"/_annotations.coco.json", "w") as outF:
    json.dump(data, outF, indent = 6)

# for imgName in imgPaths:
#     src = os.path.join('./testCOCO/'+folder,imgName)
#     dst = os.path.join("./testCOCO/filtered/"+folder,imgName)
#     os.replace(src, dst)



2304


In [ ]:
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)

def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation